# Summary
In this task, given the .dta data file, we will create a data extracting and transforming pipeline, to generate a new data frame for further analysis. Code is implemented in Python.

## Objective
Our target data frame contains columns: ['store_id', 'month', 'active_num', 'drop_out_num', 'on_board_num', 'details'],<br\>

Explanation of what each row means: In a given month(**"month"**) of a given store(**"store_id"**), <br\>
**"active_num"** counts how many different employees are working for this store in this month;<br\>
**"drop_out_num"** counts how many employees left this store during the **last month** and never came back, never came back means they never appeared in the records of this store after leaving;<br\>
**"on_board_num"** counts how many **new employees** joined this store during the **last month**, new means they haven't appeared in the records of this store before.<br\>
**"details"** shows exactly who dropped out and newly entered by their employee id.

## A few notes
1. We will drop the data of the first month of each store, since we do not have previous data.
2. We ingored 7 rows in the raw data, since their corresponding "month" is null.


## Data  Preprocessing

Import a few packages at first.

In [1]:
import pandas as pd
from datetime import datetime
from pandas import Timestamp  #原来这个是pandas里面的

Read the raw stata file "personnel_data.dta" as a Pandas dataframe.

In [2]:
df=pd.read_stata('Y://Data//ra//labor2//personnel_data.dta')

To look more clear and reduce complexity, we create a new dataframe that only includes columns that we might use in this task.

In [3]:
dfuse = df.loc[:,['id_pro','month', 'store_num', 'store_date', 'entry_date','hire_date']]

Transform the employee id column from float to int.

In [5]:
# transform id
new_id=[int(ele) for ele in dfuse['id_pro']]
dfuse['id_pro']=new_id

Filter rows whose "month" column is null.

In [6]:
#take care of this missing values later on
dfuse[dfuse['month'].isnull()]

,id_pro,month,store_num,store_date,entry_date,hire_date
16736,5391576,NaT,145,2015-10-07,2015-10-07,2015-10-07
16796,5393424,NaT,515,2015-10-15,2015-10-15,2015-10-15
16807,5393838,NaT,537,2015-10-16,2015-10-16,2015-10-16
18491,5473824,NaT,516,2016-01-10,2016-01-10,2016-01-10
18526,5474222,NaT,9605,2016-01-16,2016-01-16,2016-01-16
18562,5475698,NaT,516,2016-01-23,2016-01-23,2016-01-23
20351,5548628,NaT,139,2016-06-20,2016-06-20,2016-06-20


## Main Process
First we created an empty dataframe **target_df**, and we designed 2 loops: iteration through each month (inner loop), and iteration through each store (outer loop). As we iterate through the 2 loop, we will sequentially fill in the target dataframe.

The minimum unit we deal with is the subdataframe of records for each month in each store: **month_df**.
**emp_thism** is the set of employees that are actively working in this month, thus the number of active employees is just the length of this set. <br\>

To get this drop-out number and details, all we need to do is 2 things: 
1. First check which employee appeared in the last month and not appear this month, denote this set as **potential_drop**, it was calculated by using set difference operation between 2 months' employee id (**emp_lastm** and **emp_thism**).
2. In **potential_drop** set, get those employee id that never appeared afterwards, and this is our **drop_set**. How do we do this? We aggregated all unique employee id that appeared after this month(**empafter**), and do a set difference operation.

The on-board number calculation is just symmetric as the drop-out calculation, thus I'll not go into details here.

In [28]:
target_df=pd.DataFrame(columns=['store_id', 'month','active_num', 'drop_out_num', 'on_board_num', 'details'])
index=0
storenum=set(dfuse['store_num']) #get a unique list of store id

#----------------------------------- outer loop ----------------------------------------------
for store in storenum:   
    
    # Get the subdataframe of each store.
    subdf=dfuse[dfuse['store_num']==store]
    monthlist=sorted(set(subdf['month']))
    monthlist=[ele for ele in monthlist if ele.month in range(1,13)] # not include NaT

#----------------------------------- inner loop -----------------------------------------------
    for i in range(len(monthlist)): 
        
        # get subdataframe of each month in each store
        monthdf=subdf[subdf['month']==monthlist[i]]
               
        emp_thism=set(monthdf['id_pro']) # set of active employees
        active_num=len(emp_thism) # active employees at this time
        
        # first month, do nothing but skip
        if i==0:
            continue
            
        # not the first month, start working
        else:
            lastmonthdf=subdf[subdf['month']==monthlist[i-1]] #subdataframe of last month
            emp_lastm=set(lastmonthdf['id_pro'])  #employees of last month
            potential_drop = emp_lastm.difference(emp_thism) 
            potential_on   = emp_thism.difference(emp_lastm) 
            empafter  = set(subdf[subdf['month'] > monthlist[i]]["id_pro"])
            empbefore = set(subdf[subdf['month'] < monthlist[i]]["id_pro"])
            drop_set = potential_drop.difference(empafter)
            on_set = potential_on.difference(empbefore)
            message={}
            message['drop out detail']=drop_set
            message['on board detail']=on_set      
            
        # fill in one row of our target dataframe, and go to next iteration
        target_df.loc[index,:]=[store, monthlist[i],active_num, len(drop_set), len(on_set), message]
        index+=1 

At last, save the target data frame as a csv file.

In [26]:
target_df.to_csv("Y://gap_result.csv")